# OpenStreetMap Data Case Study


## Map Area

**Tbilisis Georgia**

-  https://www.openstreetmap.org/relation/4479704

This map is of my hometown the capital of Georgia, so I’m more interested to see what database querying reveals, and I’d like an opportunity to contribute to its improvement on OpenStreetMap.org.

## Problems Encountered in the Map

After initially downloading a small sample size of the Tbilisi area and running it against a provisional data.py file, I noticed problems with the data, which I will discuss in the following order:

-  Overabbreviated and inconsistency in street definition  (St., st, Ave, av etc ) 
-  For some streets the names are with Georgian alphabet only others with Russian only (small fraction) and for some others bothe English names and in brakets, Georgian. 
-  Overabbreviated and inconsitancy in street definition when they written in Georgian  (ქ. ქუჩა, გამზ. გამზირი  etc )
-  The street names appiears in names, name:en and name:ka and name:ru, but only for some of them exists all three.
-  Inconsistent postal codes (0214, 102, 118).
-  City names are written with Latin, Georgian or other alphabets, also some of them with undefined digits.

In [1]:
# creates small sample from area "Tbilisi.osm" file
#%run sample_tbilisi.py

In [2]:
# create csv files including user made corrections
#%run data.py

### Over Abbreviate Street Names


Was observed inconsistency in street definition  (St., st, Ave, av etc ) also some of them is written with different alphabets. In data.py file is created function which deals with inconsistency of street definition written in Latin. 
There are tags like addr:street:en, addr:street:ka, but only in few cases. These could be improved by adding this kind of tags for all cases.

The functions below dealing with street type definition:

<font color='red'>

def audit_street_type(street_types, street_name):
   
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
 
        if street_type in mapping.keys():
          name = re.sub(m.group(), mapping[m.group()], name)
                                         
    return name
   


def clean_street_names(name):
    
    street_types = defaultdict(set)
    audit_street_type(street_types, name)
    better_name = update_name(name, mapping)
          
    return better_name

In [3]:
# import sqlite

import sqlite3
db = sqlite3.connect("Tbilisi.db")
c = db.cursor()

## Postal Codes
Postal code strings posed a different sort of problem. City Tbilisi codes begin with 01 and contains with four digits (e.g 0179) but few years ago nearest villages joined to Tbilisi region and their postal codes is different and begins with 13 (e.g 1320). Some codes have only 3 digits, others-5. These ones were replaced with 'unknown'. Some streets codes are also incorrect as odd and even house numbers have different codes, but many cases housnumber isn't avalable.

The functions below replaces with uknown the postal codes which contains only 3 or 5 digits instead of 4:

<font color='red'>
def clean_postcode(code):
  
    if len(code)!=4:
        code = 'unknown'
    
    return code

<font color='black'>
Here are the top ten results, beginning with the highest count:

In [4]:
QUERY = "SELECT tags.value, COUNT(*) as count FROM \
        (SELECT * FROM nodes_tags UNION ALL  SELECT * FROM ways_tags) \
        tags WHERE tags.key='postcode'\
        GROUP BY tags.value ORDER BY count DESC Limit 100;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'1320', 926), (u'0186', 228), (u'1323', 134), (u'0182', 126), (u'0107', 95), (u'0183', 81), (u'0160', 56), (u'0112', 47), (u'0179', 42), (u'0108', 41), (u'0105', 34), (u'0141', 31), (u'0177', 31), (u'0159', 30), (u'0164', 27), (u'0102', 26), (u'0131', 26), (u'0171', 16), (u'0154', 13), (u'0121', 12), (u'0162', 12), (u'0103', 11), (u'0101', 8), (u'0144', 8), (u'0194', 8), (u'0113', 7), (u'0178', 6), (u'0114', 5), (u'unknown', 5), (u'0100', 3), (u'0145', 3), (u'0152', 3), (u'0168', 3), (u'0192', 3), (u'3300', 3), (u'0014', 2), (u'0115', 2), (u'0118', 2), (u'0119', 2), (u'0137', 2), (u'0153', 2), (u'0163', 2), (u'0176', 2), (u'3701', 2), (u'0017', 1), (u'0135', 1), (u'0167', 1), (u'0172', 1), (u'0180', 1), (u'0190', 1), (u'0214', 1), (u'0610', 1), (u'1086', 1), (u'3311', 1)]


## Sort cities by count

The city names were written with many different ways. Typos were corrected, Georgian alphabet written names were moved into Latin, only digits were replaced with unknownm but still are some problems.

The function below cleans the city names:

<font color='red'>
def clean_cityname(name):
    
    if 'Tbi' in name or 'tbi' in name:
        name='Tbilisi'
    elif name.isdigit(): #if only digits instead of name relpace it with unknown
        name = 'Unknown'
    elif 'Martkopi' in name or 'Zemo' in name: # after Tbilisi the most common 
        return name
    else:
        for char in name:
             if translit(char, 'ka') in name:
                    temp_name=name.replace(char,translit(char, "ka", reversed=True)) # Replace Georgian characters with Latin
                    if 'bili' in name:
                        name ='Tbilisi'
                    else:
                        name = temp_name
    
        
    return name


In [5]:
QUERY = "SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL  SELECT * FROM ways_tags) \
                tags WHERE tags.key LIKE 'city' GROUP BY tags.value ORDER BY count DESC;"
c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'Tbilisi', 3583), (u'Martkopi', 1193), (u'Norio', 134), (u'Zemo Phonichala', 124), (u'\u0422\u0431\u0438\u043b\u0438\u0441\u0438', 6), (u'Kojori', 1), (u'Marjanishvili', 1), (u'Mtskheta', 1), (u'Okrokana', 1), (u'Shindisi', 1), (u'Tabakhmela', 1), (u'Tblisi', 1), (u'teleti', 1), (u'\u0421\u0435\u043b\u043e \u0412\u0430\u0448\u043b\u043e\u0432\u0430\u043d\u0438', 1)]


## Data Overview and Additional Ideas
This section contains basic statistics about the dataset, the SQL queries used to gather them, and some additional ideas about the data in context.

###  File sizes
Tbilisi.osm    ......... 89 MB

Tbilisi.db     ......... 64 MB 

nodes.csv      ......... 34 MB

nodes_tags.csv ......... 1 MB

ways.csv       ......... 2 MB

ways_nodes.csv ......... 11 MB 

ways_tags.csv  ......... 4 MB 
 
 
 
 
### Number of nodes
  

In [6]:
QUERY = "SELECT COUNT(*) FROM nodes;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(424636,)]


### Number of ways

In [7]:
QUERY = "SELECT COUNT(*) FROM ways;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(48944,)]


Number of unique users


In [8]:
QUERY = "SELECT COUNT(DISTINCT(e.uid))  FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(1222,)]


Top 10 contributing users

In [9]:
QUERY = "SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'user_16881', 179697), (u'Andro', 69503), (u'moszkva ter', 59648), (u'nikusha', 19352), (u'comandant3', 17896), (u'Davidgogishvili', 11275), (u'Irakli Ika', 10360), (u'jeffhaack', 7506), (u'gguj', 7217), (u'Ben Allard', 5744)]


Number of users appearing only once (having 1 post)

In [10]:
QUERY = "SELECT COUNT(*) FROM\
    (SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e\
     GROUP BY e.user\
     HAVING num=1) u ;"

c.execute(QUERY)
rows = c.fetchall()
print rows


[(523,)]


## Additional Data Exploration


### Top 10 appearing amenities

In [11]:
QUERY = "SELECT value, COUNT(*) as num FROM nodes_tags WHERE key='amenity' GROUP BY value ORDER BY num DESC LIMIT 10;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'restaurant', 363), (u'pharmacy', 246), (u'fuel', 206), (u'cafe', 203), (u'atm', 199), (u'bank', 176), (u'place_of_worship', 144), (u'school', 124), (u'fast_food', 103), (u'parking', 101)]


### Most popular cuisines


As a result with one look hard to rank whick cuisines is the most popular after Georgian and even this is devided several parts like:  'regional;georgian', Georgian, Khachapuri etc

In [12]:
QUERY = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN \
        (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') \
        i ON nodes_tags.id=i.id WHERE nodes_tags.key='cuisine' \
        GROUP BY nodes_tags.value ORDER BY num DESC;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'georgian', 51), (u'regional', 23), (u'italian', 9), (u'turkish', 5), (u'indian', 4), (u'international', 4), (u'chinese', 3), (u'japanese', 3), (u'Georgian', 2), (u'american', 2), (u'burger', 2), (u'french', 2), (u'khachapuri', 2), (u'lebanese', 2), (u'pizza', 2), (u'regional;georgian', 2), (u'russian', 2), (u'thai', 2), (u'Georgian, Ukrainian', 1), (u'Georgian_Russian', 1), (u'PUB', 1), (u'Uzbek', 1), (u'asian', 1), (u'bavarian', 1), (u'burger;steak_house', 1), (u'georgian;deli;buschenschank', 1), (u'german', 1), (u'grill;cake;georgian;local;coffee_shop;deli;chicken;vegetarian;barbecue', 1), (u'indian;arab', 1), (u'korean;asian', 1), (u'mediterranean;lebanese;italian', 1), (u'mexican', 1), (u'ossetian', 1), (u'regional; khinkali', 1), (u'regional;pizza;burger', 1), (u'seafood', 1), (u'steak_house;italian;pizza', 1), (u'ukrainian', 1), (u'uzbek', 1)]


### The Biggest religion

The information is not complete. Its not surprise thet Christian is most common but there are much more churches and other religions too. It could be checked if is possible from given data to fill missing data, or use other sources to get an information and add to this data. Also it could be add information that is cross-checked: e.g I know that there is Evangelical-Lutheran Church in Tbilisi at 19, Terenti Graneli Street, also historycal Synagogue at Leselidze Street etc. 

In [13]:
QUERY = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags\
         JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship')\
         i ON nodes_tags.id=i.id WHERE nodes_tags.key='religion'\
         GROUP BY nodes_tags.value ORDER BY num DESC;"

c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'christian', 108), (u'muslim', 2), (u'zoroastrian', 1)]


### Biggest pharmacy

The name of second and fourth biggest pharmacy is not available, may be some alphabetic issues here as well.

In [14]:
QUERY = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id)\
         FROM nodes_tags WHERE value='pharmacy') i ON nodes_tags.id=i.id \
         WHERE nodes_tags.key='name' GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;"


c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'PSP', 15), (u'\u10e4\u10d0\u10e0\u10db\u10d0\u10d3\u10d4\u10de\u10dd', 7), (u'GPC', 6), (u'\u10d0\u10e4\u10d7\u10d8\u10d0\u10e5\u10d8', 6), (u'Aversi', 5)]


### Top three biggest Bank

In [15]:
QUERY = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id)\
         FROM nodes_tags WHERE value='bank') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name'\
         GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 3;"


c.execute(QUERY)
rows = c.fetchall()
print rows

[(u'Liberty Bank', 11), (u'TBC Bank', 11), (u'Bank of Georgia', 10)]


As a result Tbilisi are map far from being in a good shape. Some issues were corrected but there are problems for many tag values. Improving problems will help user to visit the place he/she wanted to or have an correct information about the city. 
If import information from other source may be some other problems will pop up, also could be damaged parts of existing info or duplicate information. 